#### Import

In [5]:
import os
import sys
import datetime
import numpy as np
import pandas as pd
import h5py
import scipy
import matplotlib.pyplot as plt
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.losses import MeanSquaredError,cosine_similarity,MeanAbsoluteError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import timeseries_dataset_from_array
from scipy import spatial
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error
from scipy.sparse import csr_matrix, save_npz, load_npz
from sklearn.metrics.pairwise import cosine_similarity

#### Functions for .h5 Reading 

In [11]:
def grab_data(path):
    '''
    input: path to .h5 file
    output: resamples ADC2 Signal, IMU Signal, data_len = list of all journey length, min_samples= shortest journey (samples), min_index= shortest journey (index)
    '''
    
    # read in file 
    f = h5py.File(path, 'r')

    #create list of all journeys
    journey_list = list(f.keys())

    #creaty empty dfs for ADC2- and IMU-data 
    ADC2= pd.DataFrame()
    IMU= pd.DataFrame()
    data_len = []
    
    #go through all jounrey and grab IMU and ADC Data
    for jrn in journey_list:
        
        adc_data = f[jrn]['ADC2']['data_ADC2']
        imu_data = f[jrn]['IMU']['data_IMU']

        #select ch0 = z_acc from ADC Data 
        df_adc = pd.DataFrame()
        for col in adc_data.dtype.names:
            df_adc[col] = adc_data[col]         #here df for each journey gets stored
        ADC2[jrn] = df_adc['ch0']               #here X gets written column by column

        # resample ADC to IMU SampLe Rate
        ADC2_res = pd.DataFrame(scipy.signal.resample_poly(ADC2,100 , 20625, axis=0, window=('tukey'), padtype='constant', cval=None))
        
        #select z_acc from IMU Data 
        df_imu = pd.DataFrame()
        for col in imu_data.dtype.names:
            df_imu[col] = imu_data[col]      #here df for each journey gets stored
        IMU[jrn] = df_imu['acc_z']           #here y gets written column by column

    # count length of jounreys (IMU sample rate) and find minimum index -> other journeys needs to be shortened
    for jrn in journey_list:
        data_len.append(len(f[jrn]['IMU']['data_IMU']['acc_z'])) # Direct access on the h5-file f
    
    min_samples = min(data_len)                      # Shorttest timeseries (=journey) in session contains min_sample timepoints
    min_index = data_len.index(min_samples)      # min:index = shortest jounrey
    
    len_adc = len(ADC2_res.iloc[:,0])                
    index_list_adc = list(range(min_samples, len_adc)) # list from min_samples to last element in ADC
    ADC2_res.drop(ADC2_res.index[index_list_adc], inplace =True) # shorten Dataframe
 
    len_imu = len(IMU.iloc[:,0])
    index_list_imu = list(range(min_samples, len_imu))
    IMU.drop(IMU.index[index_list_imu], inplace =True)

    ADC2_res.columns = IMU.columns        # rename columns in ADC2_res
    
    return ADC2_res, IMU, data_len, min_samples, min_index

#### Sync Data Function(DataFrame, NP, Sparse)

In [16]:
def sync_data_df(ADC,IMU):
#input: ADC and IMU as Dataframes (session by session)
#output: synced ADC session as pd.DataFrame, sync_log = list of all shifts 
    # Error handling: wrong dimensions
    if IMU.shape != ADC.shape:
        print('Error: Dimensions of ADC data and IMU data are not matching')
        sys.exit()
        
    ADC_sync= pd.DataFrame()
    sync_log = []
    for column in ADC:
        ADC_ = ADC[column]
        IMU_ = IMU[column]
        corr = np.correlate(IMU_ , ADC_ , mode='same')
        delta = (len(corr)/2) - np.argmax(np.abs(corr))
        sync_log.append(delta)
        
        # create ADC_sync by padding with zeros and cutting of delta elements
        if delta > 0: # if delta positive, zeros will be podded at the end -> points at the beginning will be cut off
            ADC_sync_ = np.append(ADC_,np.zeros(int(delta)))[int(delta):]
        elif delta < 0: # if delta negaitve, zeros will be podded at the beginning -> points at the end will be cut off
            ADC_sync_ = np.append(np.zeros(int(np.abs(delta))),ADC_)[:-int(np.abs(delta))]
        elif delta == 0:
            ADC_sync_ = ADC_
            
        ADC_sync[column] = pd.DataFrame(ADC_sync_)
        
    return ADC_sync, sync_log

In [17]:
def sync_data_np(ADC,IMU):
#input: ADC and IMU as np.arrays (journey by journey)
#output: synced ADC journey as np.array, delta = shift
    # cross correlation of ADC and IMU Signal
    corr = np.correlate(IMU , ADC , mode='same')

    # Delta = value to shift ADC in order to sync with ADC signal
    # When signals are in sync, delta is 0 because peak of correlation functions is in the middle of the timeseries
    delta = (len(corr)/2) - np.argmax(np.abs(corr))
    delta_int = int(np.abs(delta))
                                     
    
    # create ADC_sync by padding with zeros and cutting of delta elements
    if delta > 0: # if delta positive, zeros will be podded at the end -> points at the beginning will be cut off
        
        ADC_sync = np.pad(ADC[delta_int:],((0,delta_int)),'edge')
        #ADC_sync = np.append(ADC,np.zeros(int(delta)))[int(delta):]
    elif delta < 0: # if delta negaitve, zeros will be podded at the beginning -> points at the end will be cut off
        ADC_sync = np.pad(ADC[:int(len(ADC)+delta)],((delta_int,0)),'edge')
        #ADC_sync = np.append(np.zeros(int(np.abs(delta))),ADC)[:-int(np.abs(delta))]
    elif delta == 0:
        ADC_sync = ADC
        
    return ADC_sync, delta

In [18]:
def sync_sparse(adc_sp,imu_sp,mx):
#input: ADC and IMU as sparse arrays (session by session)
#output: synced ADC session as sparse, deltas = list of all shifts 
    # grab first journey
    adc_first = adc_sp[0].data
    imu_first = imu_sp[0].data
    deltas = []

    # ceate first row by hand
    sync, delta = sync_data_np(adc_first, imu_first)
    new_adc = pad_zero(sync, mx)

    for jrn in range(adc_sp.shape[0]-1):
        sync, delta = sync_data_np(adc_sp[jrn+1].data, imu_sp[jrn+1].data)
        new_row = pad_zero(sync,mx)
        new_adc = np.vstack((new_adc, new_row))
        deltas = np.append(deltas,delta)

    # create sparse array from array                 
    adc_sync_sp = csr_matrix(new_adc)
    return adc_sync_sp, deltas

#### Filter Function (NP, Sparse)

In [4]:
# Apply Bandpass filter
from scipy.signal import butter, lfilter, sosfiltfilt

# define function for creating filter object
def butter_bandpass(lowcut, highcut, fs, order):    
#input: lowcut and highcut frequendy of the filter, fs = sampling rate, order = filter order
# output: butterworth filter parameters
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    sos = butter(order, [low, high],output='sos',btype='band')
    return sos

# define function to apply created filter to data
def butter_bandpass_filter(data, lowcut, highcut, fs, order):
# input: data = signal that gets filtered, lowcut and highcut frequendy of the filter, fs = sampling rate, order = filter order
# output: y = filtered signal
    sos = butter_bandpass(lowcut, highcut, fs, order)
    y = sosfiltfilt(sos, data)
    return y

def filt_sparse(adc,imu,low, high_a, high_i, fs, mx, ordn):
# input: adc and imu as sparse arrays (session by session), high_a/i = highcut freq for aba/imu, low = lowcut freq for both signals, mx = longest journey samples in this session, ordn = filter order
# output: filtered sparse arrays 
    # create first row of array
    # in Order to create Sparse elemtn in the end, i need to properly insert first element and start iteration from 2nd element on
    adc_filt = pad_zero(butter_bandpass_filter(adc[0].data, low, high_a, fs, order=ordn), mx)
    imu_filt = pad_zero(butter_bandpass_filter(imu[0].data, low, high_i, fs, order=ordn), mx)

    # fill up array with all jrn of the session
    for jrn in range(adc.shape[0]-1):
        new_adc = butter_bandpass_filter(adc[jrn+1].data, low, high_a, fs, order=2)
        new_imu = butter_bandpass_filter(imu[jrn+1].data, low, high_i, fs, order=2)
        adc_filt = np.vstack((adc_filt, pad_zero(new_adc, mx)))
        imu_filt = np.vstack((imu_filt, pad_zero(new_imu, mx)))

    # create sparse array from array                 
    adc_f = csr_matrix(adc_filt)
    imu_f = csr_matrix(imu_filt)

    return adc_f,imu_f

#### Quarter Car Function (from B.B.)

In [7]:
def quad_car(f, m_s, m_us, k_s, k_us, c):
    """
    This function models the impulse response of a 
    single degree-of freedom oscillator in the frequency domain.
    Formula is taken form Schenkendorf et al. (2016; "Improved Railway Track 
    Irregularities Classification by a Model Inversion Approach"; Formulas (4-8))
    """
    #initialisation:
    w = 2*np.pi*f
    A = np.empty((4,4))
    B = np.empty((4,1))
    C = np.empty((1,4))
    G = np.empty((len(f)),dtype=complex)
    #
    A[0:4,0] =[-c/m_s, c/m_us, 1, 0]
    A[0:4,1] =[c/m_s, -c/m_us, 0, 1]
    A[0:4,2] =[-k_s/m_s, k_s/m_us, 0, 0]
    A[0:4,3] =[k_s/m_s, -(k_s+k_us)/m_us, 0, 0]
    
    B[0:4,0] = [0, k_us/m_us, 0, 0]
    
    C[0,0:4] = [-c/m_s, c/m_s, -k_s/m_s, k_s/m_s]
    D = 0
#    C[0,0:4] = [c/m_us, -c/m_us, k_s/m_us, -(k_s+k_us)/m_us]
#    D = k_us/m_us
    for wc, ww in enumerate(w):
        G[wc] = (np.matmul(np.matmul(C,np.linalg.inv(1j*ww*np.identity(4) - A)),B)+D)[0]
    return G

#### Zero Padding Function

In [3]:
def pad_zero(arr, mx):
# input: np.array, mx = desired array size 
# output: padded np.array
    delta = mx-arr.size
    #new_arr = [arr, np.zeros(delta)]
    new_arr = np.concatenate((arr, np.zeros(delta)))
    return new_arr

#### Equalize length function (NP, sparse)

In [20]:
def eq_adc_imu_np(adc, imu):
# input: adc and imu signal as np.arrays (journey by journey)
# output: length equalised aba signal
    
    adcsize = adc.size
    imusize = imu.size   
    pd =  imusize-adcsize # pad width

    if pd <= 0:
        adcpd = adc[-pd:]       
        
    else:
    
        if pd%2==0: 
            adcpd = np.pad(adc, ((int(pd/2), int(pd/2))), 'edge')
        elif pd%2==1:
            adcpd = np.pad(adc, ((int(np.floor(pd/2)), int(np.floor(pd/2)+1))), 'edge')

    return adcpd

def eq_adc_imu_sparse(adc_sp,imu_sp, mx):
# input: aba and imu signals as sparse arrays (session by session), mx = longest journey duration in samples in this session
# output: length equalised aba signals
    
    # takes sparse array as input and creates new sparse array for adc with equal size 
    adc_first = adc_sp[0].data
    imu_first = imu_sp[0].data
    # create first row of array
    # in Order to create Sparse elemtn in the end, i need to properly insert first element and start iteration from 2nd element on
    #pd = pad_zero(eq_adc_imu_np(s4_adc_f[12].data,s4_imu_f[12].data),mx4)
    new_adc = pad_zero(eq_adc_imu_np(adc_first,imu_first), mx)

    for jrn in range(adc_sp.shape[0]-1):
        delta = imu_sp[jrn+1].data.size-adc_sp[jrn+1].data.size
        if delta <0:
            new_row = pad_zero(adc_sp[jrn+1].data[:delta],mx)
        else:
            new_row = pad_zero(eq_adc_imu_np(adc_sp[jrn+1].data, imu_sp[jrn+1].data),mx)

        new_adc = np.vstack((new_adc, new_row))

    # create sparse array from array                 
    adc_eq_sp = csr_matrix(new_adc)
    return adc_eq_sp

#### Scaling function (NP, sparse)

In [19]:
def scale_np(adc, imu):
# standard scale
# input: aba and imu signal as np.array
# output: scaled signals as np.array
    scaler_adc = StandardScaler().fit(adc.reshape(-1, 1))
    adc_sc = scaler_adc.transform(adc.reshape(-1, 1))

    scaler_imu = StandardScaler().fit(imu.reshape(-1, 1))
    imu_sc = scaler_imu.transform(imu.reshape(-1, 1))

    return adc_sc.reshape(-1),imu_sc.reshape(-1)

def scale_sparse(adc_sp,imu_sp, mx):
# standard scale
# input: aba and imu signal as sparse arrays
# output: scaled aba and imu signal as sparse arrays
    # takes sparse array as input and creates new sparse array for adc with equal size 
    adc_first = adc_sp[0].data
    imu_first = imu_sp[0].data
    # create first row of array
    # in Order to create Sparse elemtn in the end, i need to properly insert first element and start iteration from 2nd element on
    #pd = pad_zero(eq_adc_imu_np(s4_adc_f[12].data,s4_imu_f[12].data),mx4)
    adc_first_sc, imu_first_sc = scale_np(adc_first,imu_first)
    new_adc = pad_zero(adc_first_sc, mx)
    new_imu = pad_zero(imu_first_sc, mx)

    for jrn in range(adc_sp.shape[0]-1):
        adc_sc, imu_sc = scale_np(adc_sp[jrn+1].data,imu_sp[jrn+1].data)
        new_adc_row = pad_zero(adc_sc, mx)
        new_imu_row = pad_zero(imu_sc, mx)
        
        new_adc = np.vstack((new_adc, new_adc_row))
        new_imu = np.vstack((new_imu, new_imu_row))

    # create sparse array from array                 
    adc_sc_sp = csr_matrix(new_adc)
    imu_sc_sp = csr_matrix(new_imu)
    return adc_sc_sp, imu_sc_sp


def scale_np_max(adc, imu):
# MinMax Scale
# input: aba and imu signal as np.array
# output: scaled signals as np.array
    scaler_adc = MinMaxScaler().fit(adc.reshape(-1, 1))
    adc_sc = scaler_adc.transform(adc.reshape(-1, 1))

    scaler_imu = StandardScaler().fit(imu.reshape(-1, 1))
    imu_sc = scaler_imu.transform(imu.reshape(-1, 1))

    return adc_sc.reshape(-1),imu_sc.reshape(-1)

def scale_sparse_max(adc_sp,imu_sp, mx):
# MinMax Scale
# input: aba and imu signal as sparse arrays
# output: scaled aba and imu signal as sparse arrays
    # takes sparse array as input and creates new sparse array for adc with equal size 
    adc_first = adc_sp[0].data
    imu_first = imu_sp[0].data
    # create first row of array
    # in Order to create Sparse elemtn in the end, i need to properly insert first element and start iteration from 2nd element on
    #pd = pad_zero(eq_adc_imu_np(s4_adc_f[12].data,s4_imu_f[12].data),mx4)
    adc_first_sc, imu_first_sc = scale_np_max(adc_first,imu_first)
    new_adc = pad_zero(adc_first_sc, mx)
    new_imu = pad_zero(imu_first_sc, mx)

    for jrn in range(adc_sp.shape[0]-1):
        adc_sc, imu_sc = scale_np_max(adc_sp[jrn+1].data,imu_sp[jrn+1].data)
        new_adc_row = pad_zero(adc_sc, mx)
        new_imu_row = pad_zero(imu_sc, mx)
        
        new_adc = np.vstack((new_adc, new_adc_row))
        new_imu = np.vstack((new_imu, new_imu_row))

    # create sparse array from array                 
    adc_sc_sp = csr_matrix(new_adc)
    imu_sc_sp = csr_matrix(new_imu)
    return adc_sc_sp, imu_sc_sp

#### Convert to SparseTensor 
(from: https://stackoverflow.com/questions/40896157/scipy-sparse-csr-matrix-to-tensorflow-sparsetensor-mini-batch-gradient-descent)

In [26]:
def convert_sparse_matrix_to_sparse_tensor(X):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.SparseTensor(indices, coo.data, coo.shape)

### window functions

In [4]:
# Create Window Dataset 
# Input format: np.array
def arr_to_window(X, y, window):
# input: 2 signals X and y as np.arrays , desired window size
# output: 2 new arrays that contain structured data with desired window size 
    X_win = []
    y_win = []
    samples = len(X)
    for i in range(int(samples/window)):
        i = i*window 
        row_x = [[a] for a in X[i:i+window]] # adding in korrekt format!
        row_y = [[a] for a in y[i:i+window]] # adding in korrekt format!
        X_win.append(row_x)
        y_win.append(row_y)
            
    return np.array(X_win), np.array(y_win)
    
# 
def sparse_to_window(X_sp, y_sp, window):
# input: sparse arrays X_sp and y_sp, desired window size
# output: 2 new sparse arrays that contain structured data with desired window size 
    jrns = X_sp.shape[0]
    X_win = []
    y_win = []
    for jrn in range(jrns):
        X_ = X_sp[jrn].data    
        y_ = y_sp[jrn].data
        samples = len(X_)
        for i in range(int(np.floor(samples/window))):
            i = i*window
            row_x = [[a] for a in X_[i:i+window]] # adding in korrekt format!
            row_y = [[a] for a in y_[i:i+window]] # adding in korrekt format!
            X_win.append(row_x)
            y_win.append(row_y)
            
    return np.array(X_win), np.array(y_win)

In [21]:
def process_data(x_sp, y_sp):
# input: sparse arrays x_sp and y_sp
# output: Dynamic Time Warp Score, Pearson Correlation Coefficient, Cosine Similarity for all arrays in this sparse arrays/ sessions in this journey
    from tslearn.metrics import dtw, dtw_path
    from scipy.stats import pearsonr
    DTW = []
    CosSin = []
    Pear = []
    
    for jrn in range(x_sp.shape[0]):

        # load journey data
        x = x_sp[jrn].data.astype(float)
        y = y_sp[jrn].data.astype(float)

        # DTW Score
        if len(x)>50000:
            DTW.append(int(100))
        else: 
            DTW.append(dtw(x, y))

        # Cosine Similarity
        CosSin.append(np.dot(x,y)/(np.linalg.norm(x)*np.linalg.norm(y)))

        # Pearson Coefficient
        corr, pvalue = pearsonr(x,y)
        Pear.append(corr)
        

    return DTW,Pear,CosSin